In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("C:/Users/ishan/Desktop/ISHANAY/BU docs/Spring 2025/Financial_analytics/Project/FinancialScorePredictor_UsingSentimentAnalysis/data/technical_analysis_wrds.csv")
df["date"] = pd.to_datetime(df["date"])
df["TICKER"] = df["TICKER"].str.upper()
df["PRC"] = pd.to_numeric(df["PRC"], errors='coerce').abs()
df["CFACPR"] = pd.to_numeric(df["CFACPR"], errors='coerce')

C:\Users\ishan\AppData\Local\Temp\ipykernel_484\3754010988.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/ishan/Desktop/ISHANAY/BU docs/Spring 2025/Financial_analytics/Project/FinancialScorePredictor_UsingSentimentAnalysis/data/technical_analysis_wrds.csv")


In [2]:
df

,PERMNO,date,NCUSIP,TICKER,COMNAM,PERMCO,CUSIP,PRC,VOL,RET,CFACPR
0,10104,2021-12-27,68389X10,ORCL,ORACLE CORP,8045,68389X10,89.54000,7668988.0,-0.002006,1.0
1,10104,2021-12-28,68389X10,ORCL,ORACLE CORP,8045,68389X10,88.65000,5614472.0,-0.00994,1.0
2,10104,2021-12-29,68389X10,ORCL,ORACLE CORP,8045,68389X10,88.21000,5628531.0,-0.004963,1.0
3,10104,2021-12-30,68389X10,ORCL,ORACLE CORP,8045,68389X10,88.01000,7382356.0,-0.002267,1.0
4,10104,2021-12-31,68389X10,ORCL,ORACLE CORP,8045,68389X10,87.21000,5908158.0,-0.00909,1.0
...,...,...,...,...,...,...,...,...,...,...,...
348594,93436,2024-08-29,88160R10,TSLA,TESLA INC,53453,88160R10,206.28000,62056053.0,0.002576,1.0
348595,93436,2024-08-30,88160R10,TSLA,TESLA INC,53453,88160R10,214.11000,63024686.0,0.037958,1.0
348596,93436,2024-09-03,88160R10,TSLA,TESLA INC,53453,88160R10,210.60001,76310864.0,-0.016393,1.0
348597,93436,2024-09-04,88160R10,TSLA,TESLA INC,53453,88160R10,219.41000,80182824.0,0.041833,1.0


In [3]:
# 2. Adjust Prices
df["adj_price"] = df["PRC"] / df["CFACPR"]

# ---------------------------------------
# 3. Calculate Indicators by Ticker
# ---------------------------------------
df.sort_values(["TICKER", "date"], inplace=True)

def compute_tech_indicators(group):
    group = group.copy()
    group["ma_20"] = group["adj_price"].rolling(window=20).mean()
    group["ma_50"] = group["adj_price"].rolling(window=50).mean()
    group["ma_200"] = group["adj_price"].rolling(window=200).mean()
    
    # Trend signal based on MA20 vs MA50
    group["ma_signal"] = np.where(group["ma_20"] > group["ma_50"], "Uptrend", "Downtrend")

    # Bollinger Bands (20-day)
    rolling_mean = group["adj_price"].rolling(window=20).mean()
    rolling_std = group["adj_price"].rolling(window=20).std()
    group["bb_upper"] = rolling_mean + (2 * rolling_std)
    group["bb_lower"] = rolling_mean - (2 * rolling_std)

    # Bollinger signal
    group["bb_signal"] = np.where(group["adj_price"] > group["bb_upper"], "Overbought",
                           np.where(group["adj_price"] < group["bb_lower"], "Oversold", "Neutral"))
    group["daily_return"] = group["adj_price"].pct_change()
    return group

df = df.groupby("TICKER").apply(compute_tech_indicators).reset_index(drop=True)


C:\Users\ishan\AppData\Local\Temp\ipykernel_484\764909724.py:27: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  group["daily_return"] = group["adj_price"].pct_change()
C:\Users\ishan\AppData\Local\Temp\ipykernel_484\764909724.py:27: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  group["daily_return"] = group["adj_price"].pct_change()
C:\Users\ishan\AppData\Local\Temp\ipykernel_484\764909724.py:27: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'f

In [12]:
df.head()

,PERMNO,date,NCUSIP,TICKER,COMNAM,PERMCO,CUSIP,PRC,VOL,RET,CFACPR,adj_price,ma_20,ma_50,ma_200,ma_signal,bb_upper,bb_lower,bb_signal,daily_return
0,87432,2021-12-27,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,158.74001,951771.0,0.005957,1.0,158.74001,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,NaN
1,87432,2021-12-28,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,159.17999,764279.0,0.002772,1.0,159.17999,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,0.002772
2,87432,2021-12-29,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,160.64999,833052.0,0.009235,1.0,160.64999,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,0.009235
3,87432,2021-12-30,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,160.88000,771020.0,0.001432,1.0,160.88000,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,0.001432
4,87432,2021-12-31,00846U10,A,AGILENT TECHNOLOGIES INC,36364,00846U10,159.64999,1315542.0,-0.007646,1.0,159.64999,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,-0.007646


In [5]:
# ---------------------------------------
# 4. Calculate Sharpe Ratio
# ---------------------------------------
sharpe_df = df.groupby("TICKER")["daily_return"].agg(
    lambda r: ((r.mean() - 0.042 / 252) / r.std()) * np.sqrt(252)
).reset_index(name="sharpe_ratio")

In [6]:
sharpe_df.head()

,TICKER,sharpe_ratio
0,A,-0.159643
1,AA,-0.239262
2,AAL,-0.284250
3,AAON,0.572435
4,AAPL,0.268493


In [7]:
# 5. Keep Latest Row Per Ticker
latest = df.sort_values("date").groupby("TICKER").tail(1)
latest = latest.merge(sharpe_df, on="TICKER", how="left")

In [8]:
latest.head()

,PERMNO,date,NCUSIP,TICKER,COMNAM,PERMCO,CUSIP,PRC,VOL,RET,...,adj_price,ma_20,ma_50,ma_200,ma_signal,bb_upper,bb_lower,bb_signal,daily_return,sharpe_ratio
0,89017,2022-01-07,G9662910,WLTW,WILLIS TOWERS WATSON PUB LTD CO,41694,G9662910,231.56000,1209989.0,-0.011863,...,231.56000,NaN,NaN,NaN,Downtrend,NaN,NaN,Neutral,-0.011863,-7.533638
1,32803,2022-03-14,43610610,HFC,HOLLYFRONTIER CORP,20922,40394910,36.39000,4784780.0,0.019899,...,36.39000,33.2385,34.491400,NaN,Downtrend,39.521883,26.955117,Neutral,0.019899,1.129264
2,79338,2022-04-28,80874P10,SGMS,SCIENTIFIC GAMES CORP,7195,80874P10,58.07000,512243.0,0.02326,...,58.07000,59.2490,59.916800,NaN,Downtrend,63.306362,55.191638,Neutral,0.023260,-0.565164
3,13407,2022-06-08,30303M10,FB,META PLATFORMS INC,54084,30303M10,196.64000,22232661.0,0.00506,...,196.64000,193.2445,203.242801,NaN,Downtrend,203.670669,182.818330,Neutral,0.005060,-1.608056
4,89179,2022-06-27,03675210,ANTM,ANTHEM INC,42273,03675210,482.57999,1274535.0,0.02705,...,482.57999,481.0625,493.454600,NaN,Downtrend,522.234440,439.890560,Neutral,0.027050,0.343244


In [13]:
latest.to_csv("C:/Users/ishan/Desktop/ISHANAY/BU docs/Spring 2025/Financial_analytics/Project/FinancialScorePredictor_UsingSentimentAnalysis/data/technical_indicators_wrds_output.csv", index=False)